In [40]:
import numpy as np
import pandas as pd


In [41]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.tree import DecisionTreeClassifier

In [42]:
df = pd.read_csv('train.csv')

In [43]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Planning of pipelines

### Impute missing values--->OHE--->Scaling--->Feature Selection--->Decision Tree

In [44]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace = True)


## Train Test split

In [45]:


X_train, X_test, Y_train, Y_test = train_test_split(df.drop(columns=['Survived']),
                                                   df['Survived'],
                                                   test_size =0.2,
                                                   random_state=42)

In [48]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


### Impute Missing values

In [50]:


trf1 = ColumnTransformer([
    ('Impute_age',SimpleImputer(),[2]),
    ('impute_Embarked',SimpleImputer(strategy = 'most_frequent'),[6])
],remainder = 'passthrough')

### Encoding

In [51]:
trf2 = ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse = False, handle_unknown = 'ignore'),[1,6])
],remainder = 'passthrough')


### Scaling

In [52]:
trf3 = ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10))
])

### Feature Selection

In [53]:
trf4 = SelectKBest(score_func = chi2,k=8)

### Feature Selection

In [55]:
trf5 = DecisionTreeClassifier()

### Pipeline

In [17]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5)
])

In [21]:
pipe.fit(X_train,Y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('Impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_Embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x7c28ab5dc310>)),
                ('trf5', DecisionTreeClassifier())])

In [20]:
from sklearn import set_config
set_config(display = 'diagram')

In [22]:
y_pred = pipe.predict(X_test)

In [23]:
y_pred

array([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 0])

In [57]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test,y_pred)

0.6256983240223464

### Cross validation using Pipeline

In [58]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, X_train, Y_train, cv=5, scoring='accuracy').mean()

0.6391214419383433

### GridSearch using Pipeline( used for tunning hyperparameters)

In [30]:
params = {
    'trf5__max_depth':[1,2,3,4,5,None]
}

In [31]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe, params, cv=5, scoring='accuracy')
grid.fit(X_train, Y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('Impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_Embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('trf2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse=False),
                                                                         [1,
                                                                          6])])),
                                       ('trf3',
                                        ColumnTransformer(transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('trf4',
                                        SelectKBest(k=8,
                                                    score_func=<function chi2 at 0x7c28ab5dc310>)),
                                       ('trf5', DecisionTreeClassifier())]),
             param_grid={'trf5__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [32]:

grid.best_score_

0.6391214419383433

In [59]:

grid.best_params_

{'trf5__max_depth': 2}

### Exporting pipeline

In [60]:
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))
